### Deploy the model as a web service hosted on Azure Container Instances (ACI). 

1. Create the scoring script.
1. Prepare an inference configuration.
1. Deploy the previously trained model to the cloud.
1. Consume data sample and test the web service.

###  1. Create the scoring script.

Create the scoring script, called score.py, used by the web service call to show how to use the model.  
You must include two required functions into the scoring script:
* The `init()` function, which typically loads the model into a global object. 
    * This function is run only once when the Docker container is started. 
* The `run(input_data)` function uses the model to predict a value based on the input data. 
    * Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats are supported.

TIP: Documentation on Deploy a model to Azure Container Instances [here](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-azure-container-instance/). Advanced entry script authoring [here](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-advanced-entry-script#binary-data/).

In [ ]:
%%writefile score.py
from azureml.contrib.services.aml_request import AMLRequest, rawhttp
from azureml.contrib.services.aml_response import AMLResponse
import json, os, io
import numpy as np
import torch
import torchxrayvision as xrv
from torchvision import transforms
from torchxrayvision.datasets import normalize
import pydicom

def init():
    global modelx
    # AZUREML_MODEL_DIR is an environment variable created during deployment.
    # It is the path to the model folder (./azureml-models/$MODEL_NAME/$VERSION)
    # For multiple models, it points to the folder containing all deployed models (./azureml-models)
    model_path = os.path.join(os.getenv('AZUREML_MODEL_DIR'), 'pc-densenet-densenet-best.pt')
    # print(model_path)
    modelx = torch.load(model_path)

# TIP:  To accept raw data, use the AMLRequest class in your entry script and add the @rawhttp decorator to the run() function
#       more details in: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-advanced-entry-script
# Note that despite the fact that we trained our model on PNGs, we would like to simulate
# a scenario closer to the real world here and accept DICOMs into our score script. Here's how:
@rawhttp
def run(request):

    if request.method == 'GET':
        # For this example, just return the URL for GETs.
        respBody = str.encode(request.full_path)
        return AMLResponse(respBody, 200)

    elif request.method == 'POST':
        # For a real-world solution, you would load the data from reqBody
        # and send it to the model. Then return the response.
        try:

            # For labels definition see file: '3.Build a model/trainingscripts/padchest_config.py'
            pathologies_labels = ['Air Trapping', 'Aortic Atheromatosis', 'Aortic Elongation', 'Atelectasis',
             'Bronchiectasis', 'Cardiomegaly', 'Consolidation', 'Costophrenic Angle Blunting', 'Edema', 'Effusion',
             'Emphysema', 'Fibrosis', 'Flattened Diaphragm', 'Fracture', 'Granuloma', 'Hemidiaphragm Elevation',
             'Hernia', 'Hilar Enlargement', 'Infiltration', 'Mass', 'Nodule', 'Pleural_Thickening',
             'Pneumonia', 'Pneumothorax', 'Scoliosis', 'Tuberculosis']

            # Read DICOM and apply photometric transformations
            def read_and_rescale_image( filepath):
                dcm = pydicom.read_file(filepath)
                image = dcm.pixel_array * dcm.RescaleSlope + dcm.RescaleIntercept

                def window_image(image, wc, ww):
                    img_min = wc - ww // 2
                    img_max = wc + ww // 2
                    image[image < img_min] = img_min
                    image[image > img_max] = img_max
                    return image

                image = window_image(image, dcm.WindowCenter, dcm.WindowWidth)
                # Scales 16bit to [-1024 1024]
                image = normalize(image, maxval=65535, reshape=True)
                return image

            file_bytes = request.files["image"]

            # Note that user can define this to be any other type of image
            input_image = read_and_rescale_image(file_bytes)

            preprocess = transforms.Compose([
                xrv.datasets.XRayCenterCrop(),
                xrv.datasets.XRayResizer(224)
            ])

            input_image = preprocess(input_image)
            input_batch =  torch.from_numpy( input_image[np.newaxis,...] )
            
            with torch.no_grad():
                output = modelx(input_batch)

            index = np.argsort( output.data.cpu().numpy() )
            probability = torch.nn.functional.softmax(output[0], dim=0).data.cpu().numpy()

            #Return the result
            return {"top_three_labels":  [pathologies_labels[index[0][-1]], pathologies_labels[index[0][-2]], pathologies_labels[index[0][-3]] ],
                "probability":[ round(probability[index[0][-1]]*100,2), round(probability[index[0][-2]]*100,2), round(probability[index[0][-3]]*100,2) ] }

        except Exception as e:
            result = str(e)
            # return error message back to the client
            return AMLResponse(json.dumps({"error": result}), 200)

    else:
        return AMLResponse("bad request", 500)

### 2. Prepare an inference configuration.
   * Create an environment object
   * Create inference configuration to deploy the model as a web service using:
      * The scoring file (`score.py`)
         *  Use [AMLRequest](https://docs.microsoft.com/en-us/python/api/azureml-contrib-services/azureml.contrib.services.aml_request?view=azure-ml-py) and [AMLResponse](https://docs.microsoft.com/en-us/python/api/azureml-contrib-services/azureml.contrib.services.aml_response.amlresponse?view=azure-ml-py) classes to access RAW data

In [ ]:
%%time
import uuid
from azureml.core.webservice import Webservice
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from azureml.core.environment import Environment
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.environment import CondaDependencies

# Connect to workspace
from azureml.core import Workspace
# Load workspace from config file
# The workspace is the top-level resource for Azure Machine Learning, 
# providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning.
# Documentation: https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace
ws = Workspace.from_config(path='../')
print("Workspace:",ws.name)

# We create a light weight environment for inference 
# An Environment defines Python packages, environment variables, and Docker settings that are used in machine learning experiments,
# including in data preparation, training, and deployment to a web service.
# Documentation: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment.environment?view=azure-ml-py
rsna_env = Environment(name='rsna_demo-inference')

# Set Environment:
# The Environment manages application dependencies in an Azure Machine Learning 
# Documentation: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.conda_dependencies.condadependencies?view=azure-ml-py
conda_dep = CondaDependencies()
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("azure-ml-api-sdk")
conda_dep.add_pip_package("torch")
conda_dep.add_pip_package("torchvision")
conda_dep.add_pip_package("torchxrayvision")
conda_dep.add_pip_package("pydicom")
# Add dependencies to environment 
rsna_env.python.conda_dependencies=conda_dep

# Register model:
# A model is the result of a Azure Machine learning training Run or some other model training process outside of Azure. 
# Regardless of how the model is produced, it can be registered in a workspace, where it is represented by a name and a version. 
# With the Model class, you can package models for use with Docker and deploy them as a real-time endpoint that can be used for inference requests.
# Please set the version number accordingly the number of models that you have registered.
# Documentation: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py
model = Model(ws, 'padchest', version=6)

# Set inference and ACI web service:
# The inference configuration describes how to configure the model to make predictions. 
# It references to the scoring script (entry_script) and is used to locate all the resources required for the deployment. 
# Documentation: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.inferenceconfig?view=azure-ml-py
inference_config = InferenceConfig(entry_script="score.py", environment=rsna_env)

### 3. Deploy in ACI
   Deploy the model as ACI web service. Note that this step may take about 2-5 minutes to complete

In [ ]:
# Set AciWebservice:
# The AciWebservice class represents a machine learning model deployed as a web service endpoint on Azure Container Instances
# The Inference configuration (inference_config) is an input parameter for Model deployment-related actions
# Note that we trained using a GPU cluster and we set resource_configuration=ResourceConfiguration(cpu=1, memory_in_gb=2) respectively.
# This will allow us to run inference in CPU and optimize memory. 
# Documentation: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.inferenceconfig?view=azure-ml-py
aci_config = AciWebservice.deploy_configuration(
    cpu_cores=model.resource_configuration.cpu,
    memory_gb=model.resource_configuration.memory_in_gb)

service_name = 'padchest-aci'
# Deploy:
# The model is packaged (using Docker behind the scenes) as a real-time endpoint that is later used for inference requests.
# Documentation: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.model.model?view=azure-ml-py
service = Model.deploy(workspace=ws, 
                       name=service_name, 
                       models=[model], 
                       inference_config=inference_config, 
                       deployment_config=aci_config,
                       overwrite=True)

service.wait_for_deployment(show_output=True)


### 4. Consume data sample and test the web service.
We demonstrate how to consume DICOM images:
* We trained our model from PNG files with 16 bits pixel depth. 
* To test the web service, we will send a DICOM file (16 bits).
    * We will apply the image normalization implemented in the scoring script.

To try out the model you would need a sample DICOM image. In order to obtain one, we recommend that you use one of the PADCHEST images you trained on and use the provided `png2dcm.py` script to generate a DICOM file out of it. You can also try using your own DICOM!

In [ ]:
import pydicom
import matplotlib.pylab as plt

# Visualize converted DICOM file from the corresponding PNG file
test_file = "./sample_dicom.dcm"
dcm = pydicom.read_file(test_file)
print(dcm)
plt.imshow(dcm.pixel_array, cmap=plt.cm.bone)

Now that the model is deployed we can get the scoring web service's HTTP endpoint, which accepts REST client calls. 

In [ ]:
import requests
from azureml.core.webservice import Webservice
import numpy as np

# Webservice constructor is used to retrieve a cloud representation of a Webservice
# object associated with the provided Workspace
# Documentation: https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.webservice(class)?view=azure-ml-py
service = Webservice(name='padchest-aci', workspace=ws)

# Get the web service HTTP endpoint.
# This endpoint can be shared with anyone who wants to test the web service or integrate it into an application.
uri = service.scoring_uri
print(uri)

files = {'image': open(test_file, 'rb').read()}

# Send the DICOM as a raw HTTP request and obtain results from endpoint.
response = requests.post(uri, files=files)
print("output:", response.content)